Using Amazon's sales activity along with GDP --- Ensemble Model

In [ ]:
%%capture
!pip install pycaret[full]

In [ ]:
import pandas as pd
import numpy as np 
from pycaret.regression import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv',index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv',index_col='row_id')

GDP Dataset Year wise for Finaland, Norway and Sweden

In [ ]:
gdp_df = pd.read_csv('../input/gdp-data/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')

In [ ]:
amazon = pd.read_csv('../input/amazonn/multiTimeline (2).csv')
temp = amazon.index.tolist()
amzn = []
for i in range(1,len(temp)):
    amzn.append([temp[i][0],temp[i][1],temp[i][2],amazon.iloc[i,0]])

Creating Monthly sales Activity country ,( Month, Year ) wise and saving it in hashmap.. Working on Daily sales Activity currently using Amazon's sales activity only

In [ ]:
swed = {}
finl = {}
norw = {}

In [ ]:
for i in range(len(amzn)):
    if (amzn[i][0][:4],amzn[i][0][5:7]) not in swed:
        swed[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][1]
        finl[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][2]
        norw[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][3]
    

In [ ]:
finl

Based on amazon's sales activity during that period adding it to our test and train Data

In [ ]:
gdp_df.columns = ['year', 'Finland', 'Norway', 'Sweden']

gdp_df = pd.melt(gdp_df, id_vars=['year'], value_vars=['Finland', 'Norway', 'Sweden'], var_name='country', value_name='gdp')
gdp_df['year'] = 'Y' + gdp_df['year'].astype(str)

In [ ]:
def pre_process(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    df['month'] = df['date'].dt.month
    df['year'] = 'Y'+df['date'].dt.year.astype(str)
    df['quarter'] = 'Q'+df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    #df['Newyear'] = 0
    #df.loc[(df.date.dt.dayofyear >= 300 ) | (df.date.dt.dayofyear <30)]['Newear'] = 1
    #df['Newyearb'] = df['dayofyear'] > 300
    #df['Newyeara'] = df['dayofyear'] < 30
    df.drop(columns=['date'],inplace=True)

pre_process(train)
pre_process(test)

#combining GDP Dataset
train = train.merge(gdp_df, how='left', on=['year', 'country'])
test = test.merge(gdp_df, how='left', on=['year', 'country'])

#initialize new column for amazon data
train['amazon'] = 0
test['amazon'] = 0

for i in range(len(train)):
    if train.iloc[i,0] == 'Sweden':
        train.amazon.iloc[i] = int(swed[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 
    elif train.iloc[i,0] == 'Finland':
        train.amazon.iloc[i] = int(finl[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 
    elif train.iloc[i,0] == 'Norway':
        train.amazon.iloc[i] = int(norw[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 

        
        
for i in range(len(test)):
    if test.iloc[i,0] == 'Sweden':
        test.amazon.iloc[i] = int(swed[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))])
    elif test.iloc[i,0] == 'Finland':
        test.amazon.iloc[i] = int(finl[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))])
    elif test.iloc[i,0] == 'Norway':
        test.amazon.iloc[i] = int(norw[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))]) 


In [ ]:
train['amazon'] = train['amazon'].astype('int') 
test['amazon'] = test['amazon'].astype('int') 
train.info(), test.info()

In [ ]:
train.head()

In [ ]:
impactcalc = {}

for i in range(len(train)):
    if (train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i]) not in impactcalc:
        impactcalc[(train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i])] = [train.num_sold.iloc[i]]
    else:
        impactcalc[(train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i])].append(train.num_sold.iloc[i])

In [ ]:
impact_mean = {}
for i in impactcalc.keys():
    impact_mean[i] = sum(impactcalc[i])/len(impactcalc[i])

In [ ]:
impact_mean
train['impact'] = 0
for i in range(len(train)):
    train.impact.iloc[i] = train.num_sold.iloc[i]/impact_mean[(train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i])]

In [ ]:
lastyear = {}
for i in range(len(train)):
    if (train.country.iloc[i],'Y'+str(int(str(train.year.iloc[i])[1:])+1),train.iloc[i,2], train.store.iloc[i], train.iloc[i,9]) in lastyear:
        print("oops", (train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i], train.iloc[i,9]))
    lastyear[(train.country.iloc[i],'Y'+str(int(str(train.year.iloc[i])[1:])+1),train.iloc[i,2], train.store.iloc[i], train.iloc[i,9])] = train.impact.iloc[i]

In [ ]:
train.head()

In [ ]:
# Credit to https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def SMAPE(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    if np.mean(diff)>10:
        return 4*np.mean(diff)
    return np.mean(diff)

In [ ]:
detail_impact = {}
for i in range(len(train)):
    if (train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i], train.iloc[i,9]) in detail_impact:
        print("oops", (train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i], train.iloc[i,9]))
    detail_impact[(train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i], train.iloc[i,9])] = train.impact.iloc[i]

In [ ]:
detail_impact

In [ ]:
testimp = {}
startidx = 0
for i in range(len(train)):
    if train.year.iloc[i] == 'Y2015':
        startidx = i
    else:
        train.impact.iloc[i] = lastyear[(train.country.iloc[i],train.year.iloc[i],train.iloc[i,2], train.store.iloc[i],train.iloc[i,9])]
        #train.impact.iloc[i] = detail_impact[(train.country.iloc[i],'Y'+str(int(str(train.year.iloc[i])[1:])-1),train.iloc[i,2], train.store.iloc[i],train.iloc[i,9])]

In [ ]:
test['impact'] = 0
for i in range(len(test)):
    test.impact.iloc[i] = lastyear[(test.country.iloc[i],test.year.iloc[i],test.iloc[i,2], test.store.iloc[i],test.iloc[i,8])]
    #test.impact.iloc[i] = detail_impact[(test.country.iloc[i],'Y2018',test.iloc[i,2], test.store.iloc[i],test.iloc[i,8])] * 1.05

In [ ]:
test.head()

In [ ]:
reg = setup(data = train.iloc[0:],
            target = 'num_sold',
      transform_target=True,
      transform_target_method='box-cox',  # https://www.statisticshowto.com/box-cox-transformation/
      silent=True,
      feature_interaction = True,
      normalize=True,
      normalize_method='maxabs',
      session_id = 42,  # It is equivalent to 'random_state' in scikit-learn.
      data_split_shuffle = False,  # When set to False, prevents shuffling of rows during 'train_test_split'.
      fold=10,
           use_gpu=True)

In [ ]:
models(internal=True)[['Name', 'GPU Enabled']]
#tuned_dt_skopt = tune_model(dt, search_library = 'scikit-optimize')

In [ ]:
add_metric('SMAPE', 'SMAPE', SMAPE, greater_is_better = False)
top =compare_models(sort = 'SMAPE',n_select = 2)

In [ ]:
evaluate_model(lgbm)

In [ ]:
lgbm =compare_models(sort = 'SMAPE',include=['lightgbm'])
rf =compare_models(sort = 'SMAPE',include=['rf'])

In [ ]:
blend = blend_models(top)
predict_model(blend)

In [ ]:
final_blend = finalize_model(blend)
predict_model(final_blend)

In [ ]:
preds = predict_model(final_blend, data=test)
sub = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
for i in range(len(sub)):
    sub.iloc[i,0] = i+26298
sub.to_csv('submission1.csv', index = False)
print(sub.head(),sub.describe())

In [ ]:
sub.to_csv('submission36.csv', index = False)
print(sub.head(),sub.describe())

In [ ]:
preds = predict_model(lgbm, data=test)
sub1 = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
preds = predict_model(rf, data=test)
sub2 = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
for i in range(len(sub1)):
    sub1.iloc[i,0] = i+26298
    sub1.iloc[i,1] = (sub1.iloc[i,1]+sub2.iloc[i,1])/2
sub1.to_csv('combination1.csv', index = False)
print(sub1.head(),sub1.describe())

In [ ]:
preds = predict_model(lgbm, data=test)
sub1 = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
for i in range(len(sub1)):
    sub1.iloc[i,0] = i+26298
sub1.to_csv('lgbm.csv', index = False)
print(sub1.head(),sub1.describe())